In [2]:
import pandas as pd
import numpy as np
import arcpy

OVERWRITE = True

# TODO: There's some data that is inclued as a sort of separate table with the .xlsx file, that we are just throwing away at present
# TODO: Handle combined precincts
# TODO: Delete superfluous fields from derived_data.gdb/blocks

In [3]:
def to_appropriate_column_name(s):
    s = s.lower().replace(' ', '_')
    s = ''.join([ch for ch in s if ch.isalnum() or ch == '_'])
    return s

## Clean the population data

In [4]:
pop_df = pd.read_excel('source_data/2010_Pop_Block_County.xls', sheetname='San Francisco County', header=4)
pop_df = pop_df.drop('BLOCKS', axis='index').reset_index()
pop_df = pop_df.rename(columns={'index': 'block_str'})

records = []
for ix, row in pop_df.iterrows():
    splits = row['block_str'].split(', ')
    record = row[[x for x in row.index if x != 'block_str']].to_dict()
    record['block'] = str(splits[0].split('Block ')[-1])
    record['block_group'] = str(splits[1].split('Block Group ')[-1])
    census_tract = '0' + str(splits[2].split('Census Tract ')[-1].replace('.', ''))
    census_tract += '0' * (6-len(census_tract))  # Even if it doesn't have a decimal part, needs to be 6 characters
    record['census_tract'] = census_tract
    records.append(record)
pop_df = pd.DataFrame(records)

pop_df.columns = [to_appropriate_column_name(x) for x in pop_df.columns]
str_columns = ['block', 'block_group', 'census_tract']
nonstr_columns = [x for x in pop_df.columns if x not in str_columns]
pop_df.loc[:, nonstr_columns] = pop_df[nonstr_columns].astype(int)
pop_df = pop_df[str_columns + nonstr_columns]
pop_df.to_csv('derived_data/SF_2010_pop_block.txt')
pop_df.head()

,block,block_group,census_tract,american_indian_and_alaska_native,asian,black_or_african_american,hispanic_or_latino,native_hawaiian_and_other_pacific_islander,one_race_total,some_other_race,total_population,total_population_not_hispanic_or_latino,two_or_more_races,white
0,1000,1,010100,0,0,0,0,0,0,0,0,0,0,0
1,1001,1,010100,0,4,3,1,0,44,1,44,43,0,36
2,1002,1,010100,0,0,0,0,0,0,0,0,0,0,0
3,1003,1,010100,0,0,0,0,0,0,0,0,0,0,0
4,1004,1,010100,0,0,1,0,0,1,0,1,1,0,0


## Clean and split the election data

In [5]:
vote_dfs = pd.read_excel('source_data/20161206_sov.xlsx', sheetname=None, header=3)

# Since the sheet names get cut off, we can fix them using the Contents tab
contents = vote_dfs['Contents'].copy()
contents.columns = ['key', 'name']
contents = contents.iloc[1:]
contents['key'] = contents['key'].astype(int)
contents = contents.set_index('key').to_dict(orient='index')

fixed_names = {}
for cut_name in vote_dfs:
    if cut_name == 'Contents':
        continue
    key = int(cut_name[:3])
    if key in contents:
        prefix = cut_name[:6]
        postfix = contents[key]['name']
        fixed_names[prefix + postfix] = vote_dfs[cut_name]

In [9]:
for name, df in fixed_names.items():
    if name == 'Contents':
        continue
    df = df[df['PrecinctName'].apply(lambda x: str(x).startswith('Pct '))]
    df.columns = [to_appropriate_column_name(x.replace('(%)', 'Percent')) for x in df.columns]
    df.to_csv('derived_data/{}.txt'.format(name), encoding='utf-8', index=False)

## Make a geodatabase from the election CSVs

In [11]:
arcpy.env.workspace = 'C:\Users\Charles\Documents\ArcGIS\SF_election_2016'
if OVERWRITE:
    arcpy.Delete_management('SF_vote_2016.gdb')
arcpy.CreateFileGDB_management('.', 'SF_vote_2016.gdb')
arcpy.TableToGeodatabase_conversion(
    ['derived_data\{}.txt'.format(x) for x in fixed_names.keys()],
    'SF_vote_2016.gdb')

<Result 'C:\\Users\\Charles\\Documents\\ArcGIS\\SF_election_2016\\SF_vote_2016.gdb'>

In [12]:
arcpy.env.workspace = 'C:\Users\Charles\Documents\ArcGIS\SF_election_2016\SF_vote_2016.gdb'
table_names = arcpy.ListTables()

NO_TOUCH = ['OBJECTID', 'precinctname', 'reportingtype', 'precinctid']
FLOAT = ['turnout_percent']

for table in sorted(arcpy.ListTables()):
    for field in arcpy.ListFields(table):
        if field.name not in NO_TOUCH:
            original_name = field.name
            temp_name = field.name[:5] + '_temp'
            arcpy.AddField_management(table, temp_name, 'FLOAT' if original_name in FLOAT else 'LONG')
            arcpy.CalculateField_management(table, temp_name, u'!{}!'.format(original_name), "PYTHON_9.3")
            arcpy.DeleteField_management(table, original_name)
            arcpy.AlterField_management(table, temp_name, to_appropriate_column_name(original_name[:31]))
    print('{} complete'.format(table))
    arcpy.Rename_management(table, table + '_reporting_type')

T100___President_and_Vice_President complete
T105___U_S__Senator complete
T110___U_S__Representative__District_12 complete
T115___U_S__Representative__District_13 complete
T120___U_S__Representative__District_14 complete
T125___State_Senate__District_11 complete
T130___State_Assembly__District_17 complete
T135___State_Assembly__District_19 complete
T140___Board_of_Supervisors__District_1 complete
T145___Board_of_Supervisors__District_3 complete
T150___Board_of_Supervisors__District_5 complete
T155___Board_of_Supervisors__District_7 complete
T160___Board_of_Supervisors__District_9 complete
T165___Board_of_Supervisors__District_11 complete
T170___Superior_Court_Judge__Seat_7 complete
T175___Member__Board_of_Education complete
T180___Member__Community_College_Board complete
T185___BART_Director__District_7 complete
T190___BART_Director__District_9 complete
T195___State_Proposition_51 complete
T200___State_Proposition_52 complete
T205___State_Proposition_53 complete
T210___State_Propositio

## Create precinct-level summaries of the election data
The election data is, by default, given at the level of [precinct, ReportingType], i.e. there are 2 rows for each precinct. We often want just the total number of votes by precinct, irrespective of reporting type, so we summarize appropriately.

In [36]:
agg_methods = {
    'precincts': 'MIN',
    'registration': 'MIN'
}
no_agg = ['OBJECTID', 'precinctname', 'reportingtype', 'precinctid', 'turnout_percent']

table_list = sorted(arcpy.ListTables())
for table in table_list:
    summary_table = 'precinct_summary_' + table
    if OVERWRITE and summary_table in table_list:
        arcpy.Delete_management(summary_table)        
    arcpy.Statistics_analysis(
        table,
        summary_table,
        [[x.name, agg_methods.get(x.name, 'SUM')] for x in arcpy.ListFields(table) if x.name not in no_agg],
        'precinctid'
    )
    for field in arcpy.ListFields(summary_table):
        if field.name == 'OBJECTID':
            continue
        if field.name.startswith('MIN_') or field.name.startswith('SUM_'):
            fname = field.name[4:]
        else:
            fname = field.name
        try:
            arcpy.AlterField_management(summary_table, field.name, to_appropriate_column_name(fname[:31]))
        except:
            print(summary_table)
            print(field.name)
            print(to_appropriate_column_name(fname[:31]))
            raise

precinct_summary_precinct_summary_T100___President_and_Vice_President_reporting_type
SUM_FREQUENCY
frequency


ExecuteError: ERROR 001600: Failed to alter field name.
Failed to execute (AlterField).


## Spatial join blocks to precincts

In [114]:
arcpy.env.workspace = 'C:\Users\Charles\Documents\ArcGIS\SF_election_2016'
if OVERWRITE:
    arcpy.Delete_management('source_data.gdb')
    arcpy.Delete_management('derived_data.gdb')
arcpy.CreateFileGDB_management('.', 'source_data.gdb')
arcpy.CreateFileGDB_management('.', 'derived_data.gdb')
arcpy.FeatureClassToGeodatabase_conversion(
    [
        'source_data\\Census 2010- Blocks for San Francisco/geo_export_72486aab-d116-414f-895d-9a7fbaa2a42c.shp',
        'source_data\\2012lines\\2012lines\\SF_DOE_Precincts_20120702.shp',
        'source_data\\2012lines\\2012lines\\SF_BOS_20120702_nowater.shp',
    ],
    'source_data.gdb')
arcpy.TableToGeodatabase_conversion(
    [
        'derived_data\\SF_2010_pop_block.txt',
    ],
    'source_data.gdb')

arcpy.SpatialJoin_analysis(
    u'source_data.gdb\\geo_export_72486aab_d116_414f_895d_9a7fbaa2a42c',
    'source_data.gdb\\SF_DOE_Precincts_20120702',
    'derived_data.gdb\\blocks_join_precincts'
)
# Since some blocks don't fit in a precicnt, we at least want them to have a distict
# all_fields = arcpy.ListFields('derived_data.gdb/blocks_join_precincts')
# keep_fields = ['OBJECTID', 'Shape', 'Join_Count', 'BARTDist', 'AssemDist', 'CongDist', 'NeighRep', 'PREC_2012',
#               'PREC 2010']
# filtered_fields = [x.name for x in all_fields if x.name.endswith('10') or x.name in keep_fields] + ['DISTRICT']
arcpy.SpatialJoin_analysis(
    'derived_data.gdb/blocks_join_precincts',
    'source_data.gdb/SF_BOS_20120702_nowater',
    'derived_data.gdb/blocks'
)
arcpy.Delete_management('derived_data.gdb/blocks_join_precincts')

<Result 'true'>